# Hidden Markov Models

In [2]:
import numpy as np

# Hidden states and observation symbols
states = ["S1", "S2"]
observations = ["A", "B"]
obs_seq = ["A", "B", "A"]

# Define model parameters (example values)
pi = np.array([0.6, 0.4])  # Initial probabilities
A = np.array([[0.7, 0.3],  # Transition probabilities
              [0.4, 0.6]])
B = np.array([[0.5, 0.5],  # Emission probabilities
              [0.1, 0.9]])

# Map observations to indices
obs_map = {obs: i for i, obs in enumerate(observations)}
O = [obs_map[o] for o in obs_seq]


# ---------- Forward Algorithm ----------
def forward(pi, A, B, O):
    N = len(A)      # Number of states
    T = len(O)      # Length of observation sequence
    alpha = np.zeros((T, N))

    # Initialization
    alpha[0, :] = pi * B[:, O[0]]

    # Recursion
    for t in range(1, T):
        for j in range(N):
            alpha[t, j] = np.sum(alpha[t-1, :] * A[:, j]) * B[j, O[t]]

    return alpha


# ---------- Backward Algorithm ----------
def backward(pi, A, B, O):
    N = len(A)
    T = len(O)
    beta = np.zeros((T, N))

    # Initialization
    beta[T-1, :] = 1

    # Recursion
    for t in range(T-2, -1, -1):
        for i in range(N):
            beta[t, i] = np.sum(A[i, :] * B[:, O[t+1]] * beta[t+1, :])

    return beta


# ---------- Viterbi Algorithm ----------
def viterbi(pi, A, B, O):
    N = len(A)
    T = len(O)
    delta = np.zeros((T, N))
    psi = np.zeros((T, N), dtype=int)

    # Initialization
    delta[0, :] = pi * B[:, O[0]]

    # Recursion
    for t in range(1, T):
        for j in range(N):
            prob = delta[t-1, :] * A[:, j]
            psi[t, j] = np.argmax(prob)
            delta[t, j] = np.max(prob) * B[j, O[t]]

    # Backtracking
    path = np.zeros(T, dtype=int)
    path[T-1] = np.argmax(delta[T-1, :])
    for t in range(T-2, -1, -1):
        path[t] = psi[t+1, path[t+1]]

    return [states[i] for i in path]


# Run algorithms
alpha = forward(pi, A, B, O)
beta = backward(pi, A, B, O)
viterbi_path = viterbi(pi, A, B, O)

print("Forward Probabilities:\n", alpha)
print("\nBackward Probabilities:\n", beta)
print("\nMost probable hidden state sequence (Viterbi):", viterbi_path)


Forward Probabilities:
 [[0.3      0.04    ]
 [0.113    0.1026  ]
 [0.06007  0.009546]]

Backward Probabilities:
 [[0.2032 0.2164]
 [0.38   0.26  ]
 [1.     1.    ]]

Most probable hidden state sequence (Viterbi): ['S1', 'S1', 'S1']
